In [20]:
import pandas as pd
df_original = pd.read_csv('FinalHW.csv')
# you need to predict average prices of Avocados in US
# every column might contain important info, its up to you to deal with it
#you MUST use a neural network and test other algorithms such as XGboost, RF or gradientBoosting Regressor etc..
# good luck
# until 19th of December

#data set name is FinalHW. csv
#predict AveragePrice

In [21]:
df_original.head()

,Unnamed: 0,Date,AveragePrice,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [39]:
df = df_original.copy()
df.rename(index=str, columns={"4046" : "Small Hass", "4225" : "Large Hass","4770" : "XLarge Hass" }, inplace=True)

# Let's see nan values
df[df.isna().any(axis=1)]

,Unnamed: 0,Date,AveragePrice,TotalVolume,Small Hass,Large Hass,XLarge Hass,TotalBags,SmallBags,LargeBags,XLargeBags,type,year,region
58,6,2015-11-15,1.13,339860.68,263915.78,28442.45,455.97,NaN,33656.42,13390.06,0.00,conventional,2015,Atlanta
69,17,2015-08-30,1.15,NaN,275818.27,44839.12,1034.09,61281.24,47138.07,14112.50,30.67,conventional,2015,Atlanta
16135,16,2017-09-10,2.15,9103.32,NaN,2782.28,0.00,6226.66,2482.62,3744.04,0.00,organic,2017,Nashville


In [40]:
# We don't have to remove these nan values because we can calculate them using the other columns
df.loc['58', 'TotalBags'] = df.loc['58', 'SmallBags'] + df.loc['58', 'LargeBags']
df.loc['69', 'TotalVolume'] = df.loc['69', 'Small Hass'] + df.loc['69', 'Large Hass'] + df.loc['69', 'XLarge Hass']
df.loc['16135', 'Small Hass'] = df.loc['16135', 'TotalVolume'] - df.loc['16135', 'Large Hass'] - df.loc['16135', 'XLarge Hass']

In [41]:
df.sort_values(by=['TotalVolume'])

,Unnamed: 0,Date,AveragePrice,TotalVolume,Small Hass,Large Hass,XLarge Hass,TotalBags,SmallBags,LargeBags,XLargeBags,type,year,region
10381,7,2015-11-08,1.59,84.56,3.95,3.95,0.00,76.66,73.33,3.33,0.00,organic,2015,MiamiFtLauderdale
9437,51,2015-01-04,1.73,379.82,0.00,59.82,0.00,320.00,320.00,0.00,0.00,organic,2015,BuffaloRochester
13189,8,2016-10-30,1.58,385.55,8.13,47.42,0.00,330.00,330.00,0.00,0.00,organic,2016,MiamiFtLauderdale
11698,24,2015-07-12,2.05,419.98,0.00,63.42,0.00,356.56,356.56,0.00,0.00,organic,2015,Syracuse
13193,12,2016-10-02,1.49,472.82,10.50,18.99,0.00,443.33,440.00,3.33,0.00,organic,2016,MiamiFtLauderdale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5493,33,2016-05-08,0.82,46324529.70,14223304.98,17896391.60,1993645.36,12211187.76,8747756.84,3342780.83,120650.09,conventional,2016,TotalUS
8353,34,2017-05-07,1.09,47293921.60,17076650.82,13549102.59,863471.88,15804696.31,11228049.63,4324231.19,252415.49,conventional,2017,TotalUS
5506,46,2016-02-07,0.76,52288697.89,16573573.78,20470572.61,2546439.11,12698112.39,9083373.04,3373077.87,241661.48,conventional,2016,TotalUS
8366,47,2017-02-05,0.77,61034457.10,22743616.17,20328161.55,1664383.09,16298296.29,12567155.58,3618270.75,112869.96,conventional,2017,TotalUS


In [45]:
# TotalUS, West, Mil doen't give us information about each of the cities. Thus, let's remove these rows
df1 = df.copy()
total_us_rows = df.loc[df['region'] == 'TotalUS'].index
df1.drop([*total_us_rows], axis=0, inplace=True)

In [48]:
df1['region'].unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'West', 'WestTexNewMexico'], dtype=object)